### Running in Docker container on Swoose

Started Docker container with the following command:

```docker run -p 8888:8888 -v /home/sam/data/pacbio_oly/:/home/data -it bioinformatics/bioinformatics:v0 /bin/bash```

The command allows ```/home/sam/data/pacbio_oly/``` to be accessible to the Docker container.

Once  access to Jupyter Notebook over port 8888 and makes my Jupyter Notebook GitHub repo and my data files the container was started, started Jupyter Notebook with the following command inside the Docker container:

```jupyter notebook --allow-root```

This is configured in the Docker container to launch a Jupyter Notebook without a browser on port 8888.
The Docker container is running on an image created from this [Dockerfile (Git commit db26291a](https://github.com/RobertsLab/code/commit/db26291a69d909ff41caa26a479b927ea6e501c8)

In [1]:
%%bash
date

Mon Sep 18 18:15:36 UTC 2017


In [2]:
%%bash
hostname

3291cf2bcb28


In [3]:
%%bash
lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2926.129
BogoMIPS:              5851.98
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23


In [4]:
%%bash
free -mh

             total       used       free     shared    buffers     cached
Mem:           70G        68G       2.1G       232M       550M        61G
-/+ buffers/cache:       7.1G        63G
Swap:         4.7G       5.4M       4.7G


In [5]:
%%bash
pwd

/home/data


### Run [Racon v0.5.0](https://github.com/isovic/racon)

#### Racon is an "ultrafast consensus module for raw de novo genome assembly of long uncorrected reads."

#### Using as part of pipeline: minimap/miniasm/racon

In [7]:
%%bash
which racon

In [8]:
%%bash
/usr/local/bioinformatics/racon-v0.5.0/bin/racon

  /usr/local/bioinformatics/racon-v0.5.0/bin/racon [options] <reads.fastq> <overlaps.paf> <raw_contigs.fasta> <out_consensus.fasta>

Options
  Input/Output options:
    reads               STR   Reads in FASTQ format.
    alnpath             STR   Path to a MHAP file with read-to-target overlaps.
    raw                 STR   Path to the raw contig/read sequences (output from the layout step). GFA, FASTA/FASTQ or
                              SAM formats allowed.
    out                 STR   Output consensus sequence.
        --sam            -    SAM is provided instead of MHAP. The reads file will be ignored, and seq and qual fields
                              from the SAM file will be used. [false]
        --mhap           -    Overlaps are in PAF format instead of MHAP. [false]

  Algorithm:
        --bq            FLT   Threshold for the average base quality of the input reads. If a read has average BQ <
                              specified, the read will be skipped. If valu

In [9]:
%%bash
time \
/usr/local/bioinformatics/racon-v0.5.0/bin/racon \
20170911_oly_pacbio_cat.fastq \
20170911_minimap2_pacbio_oly.paf \
20170918_oly_pacbio_miniasm_reads.gfa \
20170918_oly_pacbio_racon_consensus.fasta

[18:20:50 main] Using PAF for input alignments. (20170911_minimap2_pacbio_oly.paf)
[18:20:50 main] Loading reads.
[18:23:58 main] Hashing qnames.
[18:24:05 main] Parsing the overlaps file.
[18:24:05 main] Unique overlaps will be filtered on the fly.
[Mon, 18 Sep 17 18:24:05 +0000 ERROR] #5: Unexpected value found! Could not find qname 'm170301_162825_42134_c101174162550000001823269408211762_s1_p0/27749/11100_22781' in the input contigs file! Exiting.In function: 'ParsePAF'.

[Mon, 18 Sep 17 18:24:05 +0000 ERROR] #5: Unexpected value found! Could not find qname 'm170301_162825_42134_c101174162550000001823269408211762_s1_p0/27749/11100_22781' in the input contigs file! Exiting.In function: 'ParsePAF'.
Exiting.

real	3m16.449s
user	2m28.040s
sys	0m17.592s


#### Reading through the [example on the Racon GitHub page](https://github.com/isovic/racon#consensus-from-mappingsoverlaps-in-pafmhap-format), it seems like I need to do another step with [Minimap](https://github.com/lh3/minimap2) before I can run Racon.

#### Using the default settings in Minimap will map the reads to contigs, which is needed for Racon.

In [10]:
%%bash
time \
/usr/local/bioinformatics/minimap2-2.1.1_x64-linux/minimap2 \
-t 23 \
20170911_oly_pacbio_cat.fastq 20170905_minimap2_pacibio_oly.paf > 20170918_minimap2_mapping_oly_pacbio.paf

[M::mm_idx_gen::116.151*1.50] collected minimizers
[M::mm_idx_gen::120.734*2.20] sorted minimizers
[M::main::120.735*2.20] loaded/built the index for 537523 target sequence(s)
[M::mm_mapopt_update::129.279*2.12] mid_occ = 288; max_occ = 1715
[M::mm_idx_stat] kmer size: 15; skip: 10; is_HPC: 0; #seq: 537523
[M::mm_idx_stat::131.206*2.11] distinct minimizers: 153278982 (34.77% are singletons); average occurrences: 4.884; average spacing: 5.344
[M::mm_idx_gen::263.975*1.78] collected minimizers
[M::mm_idx_gen::267.588*2.05] sorted minimizers
[M::main::267.588*2.05] loaded/built the index for 537296 target sequence(s)
[M::mm_mapopt_update::278.744*2.01] mid_occ = 260; max_occ = 1515
[M::mm_idx_stat] kmer size: 15; skip: 10; is_HPC: 0; #seq: 537296
[M::mm_idx_stat::280.708*2.01] distinct minimizers: 155901188 (34.32% are singletons); average occurrences: 4.800; average spacing: 5.345
[M::mm_idx_gen::412.938*1.82] collected minimizers
[M::mm_idx_gen::416.895*2.00] sorted minimizers
[M::main:

#### Presumbly that worked. Let's run Racon...

In [ ]:
%%bash
time \
/usr/local/bioinformatics/racon-v0.5.0/bin/racon -t 24\
20170911_oly_pacbio_cat.fastq \
20170911_minimap2_pacbio_oly.paf \
20170918_oly_pacbio_miniasm_reads.gfa \
20170918_oly_pacbio_racon_consensus.fasta